In [1]:
from urllib.request import Request, urlopen
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

In [2]:
# Create reference to main webpage that will be used to seek out various car makes and models
url = 'https://www.carspecs.us/'
response = requests.get(url)

In [3]:
# The beautiful soup result of parsing the main page. Will search for links to pursue
bsMain = BeautifulSoup(response.text,"html.parser")

In [4]:
# Indices 24 to 91 are the car links
MainLinkList = bsMain.findAll('a')
MainLinkList[24]['href']

'/cars/acura'

In [5]:
# This gets the specific make domain for appending to the main link
make = MainLinkList[24]['href']

# Appending the link for car make to the homepage link. This leads to the list of years
NewUrl = url + make[1:]

# Updated URL Request
NewResponse = requests.get(NewUrl)

# Updated Beautiful Soup of years
bsYears = BeautifulSoup(NewResponse.text,'html.parser')



In [6]:
# Find all links on the page of car years
bsYears.findAll('a')

# Find the index of the respective year. Initially I'll only consider 2018 and 2019 cars
yr2019 = [i for i, j in enumerate(bsYears.findAll('a')) if '2019' in j][0]
# yr2018 = [i for i, j in enumerate(bsYears.findAll('a')) if '2018' in j][0]

In [18]:
yr2019

25

In [7]:
# Get the year part of link to append to make like
year = bsYears.findAll('a')[yr2019]['href']

# New url with years
NewUrl = url + year[1:]

In [8]:
# Selecting the model
NewResponse = requests.get(NewUrl)

bsModels = BeautifulSoup(NewResponse.text,'html.parser')

MainLinkList = bsModels.findAll('a')

ModelLowIndex = [i for i, j in enumerate(bsYears.findAll('a')) if 'Cars' in j][0]
ModelHighIndex = [i for i, j in enumerate(bsYears.findAll('a')) if 'Privacy Policy' in j][0]

In [9]:
model = MainLinkList[ModelLowIndex + 2]['href']

NewUrl = url + model[1:]

NewUrl

'https://www.carspecs.us/cars/2019/acura/mdx'

In [10]:
# At this stage we are at the specs page and need to start reading data into a dataframe.

NewResponse = requests.get(NewUrl)

# Testing the indices using this manual split
BeautifulSoup(NewResponse.text,'html.parser').findAll(True,{'class':'pure-u-1 pure-u-md-1-2'})[44].get_text().strip().split('\r\n')

# Engine Type (index 12) does not work
# Front Brakes (index 26) does not work
# Read Brakes (index 27) does not work
# Brake Type (index 31) does not work
# TPMS (index 44) does not work
# Ends at 71.

# NOTE: I can make this work if I utilize a conditional to split the \t\t\t\ from the above values.

['TPMS', '\t\t\t\t\tTire Pressure Monitor']

In [16]:
variables = []
for i in range(72):
    variables.append(BeautifulSoup(NewResponse.text,'html.parser').findAll(True,{'class':'pure-u-1 pure-u-md-1-2'})[i].get_text().strip().split('\r\n')[0])

In [15]:
# The pd DataFrame
pd.DataFrame(columns=variables)

,4,7,"Transmission: 9-Speed Automatic -inc: Sequential SportShift w/paddle shifters, 3-mode integrated dynamics system, Electronic Gear Selector and hill start assist",All Wheel Drive,6.43 sec,22 mpg,290 hp,"6,200",267 ft-lbs.,"4,700",...,61.10 in.,57.50 in.,38.30 in.,38.50 in.,59.10 in.,57.80 in.,35.60 in.,28.10 in.,54.70 in.,40.60 in.


In [13]:
NewResponse = requests.get(NewUrl)

# Testing the indices using this manual split
BeautifulSoup(NewResponse.text,'html.parser').findAll()

[<html>
 <head>
 <meta charset="utf-8"/>
 <meta content="en-us" http-equiv="content-language"/>
 <meta content="width=device-width, initial-scale=1" name="viewport"/>
 <meta content="upgrade-insecure-requests" http-equiv="Content-Security-Policy">
 <title>2019 Acura MDX SH-AWD </title>
 <meta content="2019 Acura MDX SH-AWD  | Car specifications and features" name="description"/>
 <link href="https://www.carspecs.us/cars/2019/acura/mdx/77619" rel="canonical"/>
 <script type="application/ld+json">
 			{
 				"@context": "http://schema.org/",
 				"@type": "Vehicle",
 				"name": "2019 Acura MDX SH-AWD ",
 				"accelerationTime": {
 					"@type": "QuantitativeValue",
 					"valueReference": "0..60 mph",
 					"value": "6.43",
 					"unitCode": "SEC"
 				}
 			}
 		</script>
 <link href="/cdn/photobox.css" rel="stylesheet"/>
 <link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet"/>
 <link href="https://fonts.googleapis.com/css?family=Roboto:300,700,900" rel=